In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math
import string as str

In [79]:
question_df = pd.read_csv("america_in_one_room/Stanford_A1R_Codebook B.csv")

In [113]:
questions_extracted = question_df[question_df['Variable'].notnull()]
questions_extracted = questions_extracted[questions_extracted['Variable'].str.startswith("Q")].reset_index()
questions_extracted.iloc[0:48][['Variable','Variable Label']].to_csv("just_questions.csv")

In [81]:
study_df = pd.read_csv("america_in_one_room/Stanford_A1R_Dataset_APSR.csv")


In [82]:
post_opinion_df = study_df[study_df['POST'] == 1]

In [83]:
grouped = post_opinion_df[post_opinion_df['GROUP'] != " "]
control = post_opinion_df[post_opinion_df['GROUP'] == " "]

In [84]:
race_dict = {1:"White, non-Hispanic", 2:"Black, non-Hispanic", 3:"Other, non-Hispanic", 4:"Hispanic", 5:"2+, non-Hispanic", 6:"Asian, non-Hispanic"}
education_dict = {1:"No HS diploma", 2:"HS graduate or equivalent", 3:"Some college", 4:"BA or above"}
gender_dict = {0:"Unknown", 1:"Male", 2:"Female"}
age_categories_dict = {1:"18-29", 2:"30-44", 3:"45-59", 4:"60+"}
party_id = {1:"Democrat", 2:"Republican", 3:"Independent"}

In [85]:
grouped["RACE"] = grouped.apply(lambda x: race_dict[x['RACETHNICITY']], axis= 1)
grouped["AGEBRACKET"] = grouped.apply(lambda x: age_categories_dict[x['AGE4']], axis= 1)
grouped["EDUCATION"] = grouped.apply(lambda x: education_dict[x['EDUC4']], axis= 1)
grouped["GENDER"] = grouped.apply(lambda x: gender_dict[x['GENDER']], axis= 1)
grouped["PARTYBEFORE"] = grouped.apply(lambda x: party_id[x['PARTYID3']], axis= 1)
#grouped["PARTYAFTER"] = grouped.apply(lambda x: party_id[int(x['T2PARTYID3'])], axis= 1)

/var/folders/qw/dj7vjbyd6kb8q83q6chjb9m00000gn/T/ipykernel_16140/3404512867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped["RACE"] = grouped.apply(lambda x: race_dict[x['RACETHNICITY']], axis= 1)
/var/folders/qw/dj7vjbyd6kb8q83q6chjb9m00000gn/T/ipykernel_16140/3404512867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped["AGEBRACKET"] = grouped.apply(lambda x: age_categories_dict[x['AGE4']], axis= 1)
/var/folders/qw/dj7vjbyd6kb8q83q6chjb9m00000gn/T/ipykernel_16140/3404512867.py:3: Sett

In [116]:
clean_df = grouped[['GROUP', 'CONDITION', 'Q1', 'Q2A', 'Q2B', 'Q2C', 'Q2D', 'Q2E', 'Q2F', 'Q2G', 'Q2H', 'Q2I', 'Q3A', 'Q3B', 'Q3C', 'Q3D', 'Q3E', 'Q3F', 'Q3G', 'Q3H', 'Q4A', 'Q4B', 'Q4C', 'Q4D', 'Q4E', 'Q4F', 'Q4G', 'Q4H', 'Q4I', 'Q4J', 'Q5A', 'Q5B', 'Q5C', 'Q5D', 'Q5E', 'Q5F', 'Q5G', 'Q5H', 'Q5I', 'Q5J', 'Q6A', 'Q6B', 'Q6C', 'Q6D', 'Q6E', 'Q6F', 'Q6G', 'Q6H', 'Q6I', 'Q6J', 'T2Q1', 'T2Q2A', 'T2Q2B', 'T2Q2C', 'T2Q2D', 'T2Q2E', 'T2Q2F', 'T2Q2G', 'T2Q2H', 'T2Q2I', 'T2Q3A', 'T2Q3B', 'T2Q3C', 'T2Q3D', 'T2Q3E', 'T2Q3F', 'T2Q3G', 'T2Q3H', 'T2Q4A', 'T2Q4B', 'T2Q4C', 'T2Q4D', 'T2Q4E', 'T2Q4F', 'T2Q4G', 'T2Q4H', 'T2Q4I', 'T2Q4J', 'T2Q5A', 'T2Q5B', 'T2Q5C', 'T2Q5D', 'T2Q5E', 'T2Q5F', 'T2Q5G', 'T2Q5H', 'T2Q5I', 'T2Q5J', 'T2Q6A', 'T2Q6B', 'T2Q6C', 'T2Q6D', 'T2Q6E', 'T2Q6F', 'T2Q6G', 'T2Q6H', 'T2Q6I', 'T2Q6J', 'RACE', 'AGEBRACKET', 'EDUCATION', 'PARTYBEFORE', 'GENDER', 'AGE']].reset_index()

In [117]:
clean_df = clean_df.reset_index().rename({"level_0":"ID"}, axis='columns').drop(columns=["index"])

In [118]:
clean_df.to_csv("clean_data/cleaned_dataset.csv")